# Merging, Joining, Concatenating

![alt text](../resources/joins_veen_diagrams.png "Veen Diagrams")

In [1]:
import pandas as pd

In [8]:
week1 = pd.read_csv('../resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('../resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('../resources/Restaurant - Customers.csv')
foods = pd.read_csv('../resources/Restaurant - Foods.csv')

week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [5]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [7]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [9]:
foods.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


### 1. The pd.concat() Method

In [10]:
# combine two objects into one
# combining week1 and week2 into a new combined object

print (f'Size of week1 : {len(week1)} - Size of week2 : {len(week2)}')

Size of week1 : 250 - Size of week2 : 250


In [17]:
# letting Pandas create a new index

sales = pd.concat([week1, week2], ignore_index = True)
sales.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [26]:
# Pre-defining a multi-index. Pandas will return a multi-index dataframe

sales = pd.concat([week1, week2], keys=['week1', 'week2'])
sales.head(3)

Customer ID  Food ID
week1 0          537        9
      1           97        4
      2          658        1

In [22]:
# Selecting a specific DataFrame

sales.loc['week1'].head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [23]:
# Selecting a multi-index Dataframe

sales.loc[('week1', 2)]

Customer ID    658
Food ID          1
Name: (week1, 2), dtype: int64

### 2. The .append() Method

In [30]:
# the only difference between append and concat is that .concat is available on pd module and append is a method of a DataFrame object

week2.append(week1, ignore_index=False).shape

(500, 2)

### 3. Inner Joins

In [32]:
### merging or inner join

week1.merge(week2, how='inner', on='Customer ID').shape

(62, 3)

In [33]:
week1.merge(week2, how='inner', on='Customer ID').head (5)

,Customer ID,Food ID_x,Food ID_y
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9


In [34]:
## let's investigate why Customer ID = 155 is duplicated
## check for the entires for Customer ID = 155 in both DataFrames

cond1 = week1['Customer ID'] == 155
cond2 = week2['Customer ID'] == 155

In [35]:
week1[cond1]

,Customer ID,Food ID
4,155,9
17,155,1


In [36]:
week2[cond2]

,Customer ID,Food ID
208,155,3


In [37]:
# 2 rows on week1
# 1 row on week2
# pandas repeated the value of the previous column instead of returning null

In [45]:
# chaging suffix header

week1.merge(week2, how='inner', on='Customer ID', suffixes=(['_week1', '_week2'])).head(5)

,Customer ID,Food ID_week1,Food ID_week2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9


In [46]:
# merging by multiple columns

week1.merge(week2, how='inner', on=['Customer ID', 'Food ID'])

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


### 4. Outer Joins

In [48]:
week1.merge(week2, how='outer', on='Customer ID').head(5)

,Customer ID,Food ID_x,Food ID_y
0,537,9.0,5.0
1,97,4.0,NaN
2,658,1.0,NaN
3,202,2.0,NaN
4,155,9.0,3.0


In [50]:
# Food ID returning null on Food ID_Y 

cond1 = week2['Customer ID'] == 97

week2[cond1]

,Customer ID,Food ID


In [51]:
# Food ID returning null on Food ID_Y 

cond1 = week1['Customer ID'] == 97

week1[cond1]

,Customer ID,Food ID
1,97,4


In [53]:
# note the shape

week1.merge(week2, how='outer', on='Customer ID').shape

(454, 3)

In [55]:
# details from where the match came from

week1.merge(week2, how='outer', on='Customer ID', indicator = True).head(10)

,Customer ID,Food ID_x,Food ID_y,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both
5,155,1.0,3.0,both
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
8,503,5.0,8.0,both
9,503,5.0,9.0,both


In [57]:
merge = week1.merge(week2, how='outer', on='Customer ID', indicator = True)
merge['_merge'].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

### 5. Left Joins

week1.head(3)

In [59]:
foods.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [64]:
# column name on matches

week1.merge(foods, how='left', on='Food ID', sort=False).head(5)

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99


### 6. The left_on and right_on Parameters

In [67]:
# used when the columns names does not match
# week1
#    Customer ID
# customer
#    ID

week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


customers.head(3)

In [73]:
week2.merge(customers, how='left', left_on='Customer ID', right_on='ID').head(5)

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,688,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,813,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,495,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,189,Roger,Gordon,Male,Skilith,Operator
4,267,3,267,Matthew,Wood,Male,Agimba,Product Engineer


In [75]:
# dropping unnecessary columns 

week2.merge(customers, how='left', left_on='Customer ID', right_on='ID').drop(columns='ID').head(5)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,Roger,Gordon,Male,Skilith,Operator
4,267,3,Matthew,Wood,Male,Agimba,Product Engineer


### 7. Merging by Indexes with the left_index and right_index Parameters